In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_filer_date = dbutils.widgets.get("p_file_date")


#### Produce Constructor Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
# race_result_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_filer_date}'") 

In [0]:
race_year_list = df_column_to_list(race_result_df,"race_year")

In [0]:
from pyspark.sql.functions import col

race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))

In [0]:
display(race_result_df)

race_id,race_year,race_name,race_date,circuit_location,driver_name,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,position,file_date,created_date


In [0]:
from pyspark.sql.functions import sum, count, when, col

constructor_standings_df = race_result_df \
    .groupBy("race_year","team") \
    .agg(sum("points").alias("total_points"),count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(constructor_standings_df.filter("race_year == 2020"))

race_year,team,total_points,wins


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

final_df = constructor_standings_df.withColumn("rank",rank().over(constructor_rank_spec))

In [0]:
display(final_df.filter("race_year == 2020"))

race_year,team,total_points,wins,rank


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")

# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")

# overwrite_function(final_df, 'f1_presentation', 'constructor_standings', 'race_year')

merge_condition = "tgt.team = src.team and tgt.race_year = src.race_year"
merge_delta_data(final_df, "f1_presentation", "constructor_standings", "race_year",processed_folder_path, merge_condition)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3393798542034602>, line 8
      1 # final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")
      2 
      3 # final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")
      4 
      5 # overwrite_function(final_df, 'f1_presentation', 'constructor_standings', 'race_year')
      7 merge_condition = "tgt.team = src.team and tgt.race_year = src.race_year"
----> 8 merge_delta_data(final_df, "f1_presentation", "constructor_standings", "race_year",processed_folder_path, merge_condition)

File <command-3651007263442800>, line 6, in merge_delta_data(input_df, db_name, table_name, partition_column, folder_path, merge_condition)
      4 def merge_delta_data(input_df, db_name,table_name,partition_column,folder_path,merge_condition):
      5     if (s

In [0]:
%sql
select * from f1_presentation.constructor_standings where race_year = 2021;

In [0]:
%sql
select race_year,count(1) from f1_presentation.driver_standings
group by race_year
order by race_year desc;